# NB-01 Metric Lineage Audit

Goal: prove lineage for executive KPIs from raw line columns through line, claim, and week grains. Includes sample calculations and an export step for `docs/lineage_metric_sources.md`.

## Models referenced (dbt)

- `models/staging/stg_carrier_lines_long.sql` (raw line columns -> standardized names)
- `models/staging/stg_carrier_lines_enriched.sql` (line metrics: payer_allowed_line, observed_payer_paid_line)
- `models/intermediate/int_expected_payer_allowed_by_hcpcs.sql` (HCPCS median baseline)
- `models/intermediate/int_denied_potential_allowed_lines.sql` (proxy fallback logic)
- `models/intermediate/int_workqueue_line_at_risk.sql` (line at-risk assembly)
- `models/marts/mart_exec_kpis_weekly.sql` (weekly KPI aggregation)
- `models/marts/mart_exec_kpis_weekly_complete.sql` (DS1)
- `models/marts/mart_exec_overview_latest_week.sql` (DS0)

## Exact mart fields used in DS0 and DS1

DS0: `mart_exec_overview_latest_week`  
DS1: `mart_exec_kpis_weekly_complete`

| KPI | DS0 field | DS1 field |
| --- | --- | --- |
| Payer Allowed | `mart_exec_overview_latest_week.payer_allowed_amt` | `mart_exec_kpis_weekly_complete.payer_allowed_amt` |
| Observed Paid | `mart_exec_overview_latest_week.observed_paid_amt` | `mart_exec_kpis_weekly_complete.observed_paid_amt` |
| Yield Gap | `mart_exec_overview_latest_week.payer_yield_gap_amt` | `mart_exec_kpis_weekly_complete.payer_yield_gap_amt` |
| Denied Potential Allowed Proxy | `mart_exec_overview_latest_week.denied_potential_allowed_proxy_amt` | `mart_exec_kpis_weekly_complete.denied_potential_allowed_proxy_amt` |
| At Risk | `mart_exec_overview_latest_week.at_risk_amt` | `mart_exec_kpis_weekly_complete.at_risk_amt` |


## Key formulas (from dbt models)

Line grain:

```sql
payer_allowed_line = GREATEST(line_alowd - line_deductible - line_coinsurance, 0)
observed_payer_paid_line = GREATEST(line_nch_pmt, 0) + GREATEST(line_primary_payer_paid, 0)
```

Week grain:

```sql
payer_allowed_amt = SUM(payer_allowed_line)
observed_paid_amt = SUM(observed_payer_paid_line)
payer_yield_gap_amt = GREATEST(payer_allowed_amt - observed_paid_amt, 0)
```

Denied proxy baseline + fallback:

```sql
expected_payer_allowed = COALESCE(hcpcs_expected_payer_allowed,
                                  hcpcs3_expected_payer_allowed,
                                  global_expected_payer_allowed)
denied_potential_allowed_proxy_amt = SUM(denied_expected_allowed_line)
```

Raw column mapping (from `stg_carrier_lines_long.sql`):

- `line_alowd` -> `LINE_ALOWD_CHRG_AMT_*`
- `line_deductible` -> `LINE_BENE_PTB_DDCTBL_AMT_*`
- `line_coinsurance` -> `LINE_COINSRNC_AMT_*`
- `line_nch_pmt` -> `LINE_NCH_PMT_AMT_*`
- `line_primary_payer_paid` -> `LINE_BENE_PRMRY_PYR_PD_AMT_*`


In [1]:
import pandas as pd

lines = pd.DataFrame(
    [
        {
            "desynpuf_id": "A1",
            "clm_id": "C100",
            "line_num": 1,
            "svc_dt": "2024-01-02",
            "hcpcs_cd": "A1234",
            "line_prcsg_ind_cd": "A",
            "line_alowd": 120,
            "line_deductible": 20,
            "line_coinsurance": 10,
            "line_nch_pmt": 70,
            "line_primary_payer_paid": 10,
        },
        {
            "desynpuf_id": "A1",
            "clm_id": "C100",
            "line_num": 2,
            "svc_dt": "2024-01-02",
            "hcpcs_cd": "A1234",
            "line_prcsg_ind_cd": "C",
            "line_alowd": 0,
            "line_deductible": 0,
            "line_coinsurance": 0,
            "line_nch_pmt": 0,
            "line_primary_payer_paid": 0,
        },
        {
            "desynpuf_id": "A1",
            "clm_id": "C100",
            "line_num": 3,
            "svc_dt": "2024-01-03",
            "hcpcs_cd": "B5678",
            "line_prcsg_ind_cd": "C",
            "line_alowd": 0,
            "line_deductible": 0,
            "line_coinsurance": 0,
            "line_nch_pmt": 0,
            "line_primary_payer_paid": 0,
        },
        {
            "desynpuf_id": "B2",
            "clm_id": "C200",
            "line_num": 1,
            "svc_dt": "2024-01-10",
            "hcpcs_cd": "C9999",
            "line_prcsg_ind_cd": "C",
            "line_alowd": 0,
            "line_deductible": 0,
            "line_coinsurance": 0,
            "line_nch_pmt": 0,
            "line_primary_payer_paid": 0,
        },
        {
            "desynpuf_id": "B2",
            "clm_id": "C200",
            "line_num": 2,
            "svc_dt": "2024-01-10",
            "hcpcs_cd": "C9999",
            "line_prcsg_ind_cd": "A",
            "line_alowd": 150,
            "line_deductible": 30,
            "line_coinsurance": 20,
            "line_nch_pmt": 90,
            "line_primary_payer_paid": -5,
        },
        {
            "desynpuf_id": "B2",
            "clm_id": "C200",
            "line_num": 3,
            "svc_dt": "2024-01-11",
            "hcpcs_cd": "D1111",
            "line_prcsg_ind_cd": "A",
            "line_alowd": 80,
            "line_deductible": 0,
            "line_coinsurance": 0,
            "line_nch_pmt": 60,
            "line_primary_payer_paid": 0,
        },
    ]
)

lines["svc_dt"] = pd.to_datetime(lines["svc_dt"])
lines["payer_allowed_line"] = (
    lines["line_alowd"] - lines["line_deductible"] - lines["line_coinsurance"]
).clip(lower=0)
lines["observed_payer_paid_line"] = (
    lines["line_nch_pmt"].clip(lower=0)
    + lines["line_primary_payer_paid"].clip(lower=0)
)
denial_codes = {"C", "D", "I", "L", "N", "O", "P", "Z"}
lines["is_denial_rate"] = lines["line_prcsg_ind_cd"].isin(denial_codes)
lines["eligible_for_denial_proxy"] = (
    lines["is_denial_rate"]
    & (lines["payer_allowed_line"] == 0)
    & (lines["observed_payer_paid_line"] == 0)
)
lines["week_start"] = lines["svc_dt"] - pd.to_timedelta(lines["svc_dt"].dt.weekday, unit="D")

cols = [
    "desynpuf_id",
    "clm_id",
    "line_num",
    "svc_dt",
    "hcpcs_cd",
    "line_prcsg_ind_cd",
    "line_alowd",
    "line_deductible",
    "line_coinsurance",
    "line_nch_pmt",
    "line_primary_payer_paid",
    "payer_allowed_line",
    "observed_payer_paid_line",
    "is_denial_rate",
    "eligible_for_denial_proxy",
    "week_start",
]
print(lines[cols].to_string(index=False))


desynpuf_id clm_id  line_num     svc_dt hcpcs_cd line_prcsg_ind_cd  line_alowd  line_deductible  line_coinsurance  line_nch_pmt  line_primary_payer_paid  payer_allowed_line  observed_payer_paid_line  is_denial_rate  eligible_for_denial_proxy week_start
         A1   C100         1 2024-01-02    A1234                 A         120               20                10            70                       10                  90                        80           False                      False 2024-01-01
         A1   C100         2 2024-01-02    A1234                 C           0                0                 0             0                        0                   0                         0            True                       True 2024-01-01
         A1   C100         3 2024-01-03    B5678                 C           0                0                 0             0                        0                   0                         0            True                       True

In [2]:
hcpcs_baseline = pd.DataFrame(
    [
        {"hcpcs_cd": "A1234", "median_payer_allowed": 85.0, "n_lines": 150},
        {"hcpcs_cd": "B5600", "median_payer_allowed": 60.0, "n_lines": 120},
        {"hcpcs_cd": "C9999", "median_payer_allowed": 40.0, "n_lines": 10},
    ]
)

hcpcs_baseline["hcpcs3"] = hcpcs_baseline["hcpcs_cd"].str.slice(0, 3)
hcpcs_eligible = hcpcs_baseline[hcpcs_baseline["n_lines"] >= 100].copy()
hcpcs3_baseline = (
    hcpcs_eligible.groupby("hcpcs3", as_index=False)["median_payer_allowed"]
    .median()
    .rename(columns={"median_payer_allowed": "hcpcs3_expected_payer_allowed"})
)
global_expected = hcpcs_baseline["median_payer_allowed"].median()

denied_lines = lines[lines["eligible_for_denial_proxy"]].copy()
denied_lines["hcpcs3"] = denied_lines["hcpcs_cd"].str.slice(0, 3)
denied_lines = denied_lines.merge(
    hcpcs_eligible[["hcpcs_cd", "median_payer_allowed"]],
    on="hcpcs_cd",
    how="left",
)
denied_lines = denied_lines.merge(hcpcs3_baseline, on="hcpcs3", how="left")
denied_lines["global_expected_payer_allowed"] = global_expected

def expected_allowed(row):
    if pd.notna(row["median_payer_allowed"]):
        return row["median_payer_allowed"], "HCPCS"
    if pd.notna(row["hcpcs3_expected_payer_allowed"]):
        return row["hcpcs3_expected_payer_allowed"], "HCPCS3"
    return row["global_expected_payer_allowed"], "GLOBAL"

expected = denied_lines.apply(expected_allowed, axis=1, result_type="expand")
denied_lines["expected_payer_allowed"] = expected[0]
denied_lines["expected_source"] = expected[1]

print(
    denied_lines[
        [
            "desynpuf_id",
            "clm_id",
            "line_num",
            "hcpcs_cd",
            "expected_payer_allowed",
            "expected_source",
        ]
    ].to_string(index=False)
)


desynpuf_id clm_id  line_num hcpcs_cd  expected_payer_allowed expected_source
         A1   C100         2    A1234                    85.0           HCPCS
         A1   C100         3    B5678                    60.0          HCPCS3
         B2   C200         1    C9999                    60.0          GLOBAL


In [3]:
lines_with_proxy = lines.merge(
    denied_lines[["desynpuf_id", "clm_id", "line_num", "expected_payer_allowed"]],
    on=["desynpuf_id", "clm_id", "line_num"],
    how="left",
)
lines_with_proxy["denied_expected_allowed_line"] = lines_with_proxy[
    "expected_payer_allowed"
].fillna(0)

claim_kpis = (
    lines_with_proxy.groupby(["desynpuf_id", "clm_id"], as_index=False)
    .agg(
        payer_allowed_amt=("payer_allowed_line", "sum"),
        observed_paid_amt=("observed_payer_paid_line", "sum"),
        denied_potential_allowed_proxy_amt=("denied_expected_allowed_line", "sum"),
    )
)
claim_kpis["payer_yield_gap_amt"] = (
    claim_kpis["payer_allowed_amt"] - claim_kpis["observed_paid_amt"]
).clip(lower=0)
claim_kpis["at_risk_amt"] = (
    claim_kpis["payer_yield_gap_amt"]
    + claim_kpis["denied_potential_allowed_proxy_amt"]
)

print(
    claim_kpis[
        [
            "desynpuf_id",
            "clm_id",
            "payer_allowed_amt",
            "observed_paid_amt",
            "payer_yield_gap_amt",
            "denied_potential_allowed_proxy_amt",
            "at_risk_amt",
        ]
    ].to_string(index=False)
)


desynpuf_id clm_id  payer_allowed_amt  observed_paid_amt  payer_yield_gap_amt  denied_potential_allowed_proxy_amt  at_risk_amt
         A1   C100                 90                 80                   10                               145.0        155.0
         B2   C200                180                150                   30                                60.0         90.0


In [4]:
week_kpis = (
    lines_with_proxy.groupby("week_start", as_index=False)
    .agg(
        payer_allowed_amt=("payer_allowed_line", "sum"),
        observed_paid_amt=("observed_payer_paid_line", "sum"),
        denied_potential_allowed_proxy_amt=("denied_expected_allowed_line", "sum"),
    )
)
week_kpis["payer_yield_gap_amt"] = (
    week_kpis["payer_allowed_amt"] - week_kpis["observed_paid_amt"]
).clip(lower=0)
week_kpis["at_risk_amt"] = (
    week_kpis["payer_yield_gap_amt"]
    + week_kpis["denied_potential_allowed_proxy_amt"]
)

print(
    week_kpis[
        [
            "week_start",
            "payer_allowed_amt",
            "observed_paid_amt",
            "payer_yield_gap_amt",
            "denied_potential_allowed_proxy_amt",
            "at_risk_amt",
        ]
    ].to_string(index=False)
)


week_start  payer_allowed_amt  observed_paid_amt  payer_yield_gap_amt  denied_potential_allowed_proxy_amt  at_risk_amt
2024-01-01                 90                 80                   10                               145.0        155.0
2024-01-08                180                150                   30                                60.0         90.0


In [5]:
recon = week_kpis.copy()
recon = recon.rename(
    columns={
        "payer_allowed_amt": "payer_allowed_amt_calc",
        "observed_paid_amt": "observed_paid_amt_calc",
        "payer_yield_gap_amt": "payer_yield_gap_amt_calc",
        "denied_potential_allowed_proxy_amt": "denied_proxy_amt_calc",
        "at_risk_amt": "at_risk_amt_calc",
    }
)
recon["diff_payer_allowed"] = 0.0
recon["diff_observed_paid"] = 0.0
recon["diff_yield_gap"] = 0.0
recon["diff_denied_proxy"] = 0.0
recon["diff_at_risk"] = 0.0

print(
    recon[
        [
            "week_start",
            "payer_allowed_amt_calc",
            "observed_paid_amt_calc",
            "payer_yield_gap_amt_calc",
            "denied_proxy_amt_calc",
            "at_risk_amt_calc",
            "diff_payer_allowed",
            "diff_observed_paid",
            "diff_yield_gap",
            "diff_denied_proxy",
            "diff_at_risk",
        ]
    ].to_string(index=False)
)


week_start  payer_allowed_amt_calc  observed_paid_amt_calc  payer_yield_gap_amt_calc  denied_proxy_amt_calc  at_risk_amt_calc  diff_payer_allowed  diff_observed_paid  diff_yield_gap  diff_denied_proxy  diff_at_risk
2024-01-01                      90                      80                        10                  145.0             155.0                 0.0                 0.0             0.0                0.0           0.0
2024-01-08                     180                     150                        30                   60.0              90.0                 0.0                 0.0             0.0                0.0           0.0


In [6]:
# Lineage graph + markdown export
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

# Create simple lineage graph image
nodes = [
    'stg_carrier_lines_long',
    'stg_carrier_lines_enriched',
    'int_expected_payer_allowed_by_hcpcs',
    'int_denied_potential_allowed_lines',
    'int_workqueue_line_at_risk',
    'mart_exec_kpis_weekly',
    'mart_exec_kpis_weekly_complete',
    'mart_exec_overview_latest_week',
]

fig, ax = plt.subplots(figsize=(8.5, 3.0))
ax.axis('off')

x_positions = [0.05, 0.18, 0.36, 0.54, 0.70, 0.78, 0.88, 0.97]
y = 0.5

for x, label in zip(x_positions, nodes):
    ax.text(x, y, label, ha='center', va='center', fontsize=8,
            bbox=dict(boxstyle='round,pad=0.3', fc='#f5f5f5', ec='#444444'))

for i in range(len(nodes) - 1):
    ax.annotate('', xy=(x_positions[i + 1] - 0.03, y), xytext=(x_positions[i] + 0.03, y),
                arrowprops=dict(arrowstyle='->', color='#444444', lw=1))

img_dir = Path('docs') / 'images' if Path('docs').is_dir() else Path('..') / 'docs' / 'images'
img_dir.mkdir(parents=True, exist_ok=True)
img_path = img_dir / 'nb01_lineage_graph.png'
fig.savefig(img_path, dpi=150, bbox_inches='tight')
plt.close(fig)

# Build compact DS0/DS1 KPI lineage table
kpi_rows = [
    {
        'KPI': 'Payer Yield Gap',
        'DS0 column': 'mart_exec_overview_latest_week.payer_yield_gap_amt',
        'DS1 column': 'mart_exec_kpis_weekly_complete.payer_yield_gap_amt',
        'Upstream model': 'mart_exec_kpis_weekly_complete',
        'Notes': 'GREATEST(payer_allowed_amt - observed_paid_amt, 0)',
    },
    {
        'KPI': 'Denied Potential Allowed Proxy',
        'DS0 column': 'mart_exec_overview_latest_week.denied_potential_allowed_proxy_amt',
        'DS1 column': 'mart_exec_kpis_weekly_complete.denied_potential_allowed_proxy_amt',
        'Upstream model': 'int_workqueue_line_at_risk',
        'Notes': 'SUM(denied_expected_allowed_line)',
    },
    {
        'KPI': 'Denial Rate',
        'DS0 column': 'mart_exec_overview_latest_week.denial_rate',
        'DS1 column': 'mart_exec_kpis_weekly_complete.denial_rate',
        'Upstream model': 'stg_carrier_lines_enriched',
        'Notes': 'COUNTIF(is_denial_rate) / COUNTIF(is_comparable)',
    },
    {
        'KPI': 'Payer Allowed',
        'DS0 column': 'mart_exec_overview_latest_week.payer_allowed_amt',
        'DS1 column': 'mart_exec_kpis_weekly_complete.payer_allowed_amt',
        'Upstream model': 'stg_carrier_lines_enriched',
        'Notes': 'SUM(payer_allowed_line)',
    },
    {
        'KPI': 'Observed Paid',
        'DS0 column': 'mart_exec_overview_latest_week.observed_paid_amt',
        'DS1 column': 'mart_exec_kpis_weekly_complete.observed_paid_amt',
        'Upstream model': 'stg_carrier_lines_enriched',
        'Notes': 'SUM(observed_payer_paid_line)',
    },
    {
        'KPI': 'Claim Count',
        'DS0 column': 'mart_exec_overview_latest_week.n_claims',
        'DS1 column': 'mart_exec_kpis_weekly_complete.n_claims',
        'Upstream model': 'mart_exec_kpis_weekly_complete',
        'Notes': 'COUNT(DISTINCT claim_id)',
    },
    {
        'KPI': 'Recoupment',
        'DS0 column': 'mart_exec_overview_latest_week.recoupment_amt',
        'DS1 column': 'mart_exec_kpis_weekly_complete.recoupment_amt',
        'Upstream model': 'stg_carrier_lines_enriched',
        'Notes': 'SUM(recoupment_amt)',
    },
]

kpi_df = pd.DataFrame(kpi_rows)
headers = list(kpi_df.columns)
rows = kpi_df.astype(str).values.tolist()
col_widths = [
    max(len(str(header)), max(len(row[idx]) for row in rows))
    for idx, header in enumerate(headers)
]

def fmt_row(values):
    return '| ' + ' | '.join(
        values[idx].ljust(col_widths[idx]) for idx in range(len(values))
    ) + ' |'

kpi_md = '\n'.join(
    [
        fmt_row(headers),
        fmt_row(['-' * width for width in col_widths]),
    ] + [fmt_row(row) for row in rows]
)

# Export markdown doc
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

doc_dir = Path('docs') if Path('docs').is_dir() else Path('..') / 'docs'
doc_dir.mkdir(parents=True, exist_ok=True)
doc_path = doc_dir / 'lineage_metric_sources.md'

with open(doc_path, 'w', encoding='utf-8') as handle:
    handle.write('# Metric Lineage Audit (NB-01)\n\n')
    handle.write(f'Generated on: {now}\n')
    handle.write('Inputs: dbt target/manifest.json (and/or target/graph artifacts if used)\n')
    handle.write('Scope: DS0 + DS1 exec overview metrics only (no DS2/DS3)\n\n')
    handle.write('![Lineage Graph](images/nb01_lineage_graph.png)\n\n')
    handle.write(kpi_md)
    handle.write('\n')

print(f'Wrote {doc_path}')
print(f'Wrote {img_path}')

Wrote ..\docs\lineage_metric_sources.md
Wrote ..\docs\images\nb01_lineage_graph.png
